# NanoSafety data summary

- using aggregated search

In [89]:
import yaml
from solrscope import aa
import ipywidgets as widgets
import requests
from importlib import reload 
from solrscope import client_solr
from solrscope import client_ambit
from solrscope import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import qgrid

In [3]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
display(service_widget)

Select enanoMapper aggregated search service:


Dropdown(description='Service:', options=('https://api.ideaconsult.net/enanomapper', 'https://api.ideaconsult.…

In [105]:
if config_security!='':
    print('Enter `{}` you have received upon subscription to http://api.ideaconsult.net'.format(config_security) )
    style = {'description_width': 'initial'}
    apikey_widget=widgets.Text(
        placeholder='',
        description=config_security,
        disabled=False,
        style=style
    )
    display(apikey_widget)

Enter `X-Gravitee-Api-Key` you have received upon subscription to http://api.ideaconsult.net


Text(value='', description='X-Gravitee-Api-Key', placeholder='', style=DescriptionStyle(description_width='ini…

In [106]:
service_uri=service_widget.value
print("Sending queries to {}".format(service_uri))
facets = client_solr.Facets()

Sending queries to https://api.ideaconsult.net/nanoreg1


In [114]:
_query="*:*"
fields=["topcategory_s","endpointcategory_s","E.method_s","substanceType_s","publicname_s","reference_owner_s"]
colnames=["Z"]
colnames.extend(fields)
colnames.append("Number of data points")
print(colnames)
#["Z","Method","Material type","Data provider","Number of data points"]
_stats=[]
reload(client_solr)
def process(prefix,val,count,key,_tuple):
    if len(_tuple)==len(fields):
        _tuple = (*_tuple,val,count)
        _stats.append(_tuple)
        #print("{}".format(_tuple)) 
    
query=facets.getQuery(query=_query,facets=fields,fq="type_s:study")
#print(query)
r = client_solr.post(service_uri,query=query,auth=aa.GraviteeAuth(apikey_widget.value))
response_json=r.json()
if r.status_code==200:
    facets.parse(response_json['facets'],prefix=">",process=process)
    df = pd.DataFrame(_stats,columns=colnames).drop("Z", axis=1)
    a = annotation.DictionarySubstancetypes()
    df[ 'substanceType_s']=df[ 'substanceType_s'].apply(a.annotate)
    a = annotation.DictionaryEndpointCategory()
    df[ 'endpointcategory_term']=df[ 'endpointcategory_s'].apply(a.annotate)
    a = annotation.DictionaryAssays()
    df[ 'method_term']=df[ 'E.method_s'].apply(a.annotate)
    display(df.head())
else:
    print(r.status_code)

['Z', 'topcategory_s', 'endpointcategory_s', 'E.method_s', 'substanceType_s', 'publicname_s', 'reference_owner_s', 'Number of data points']


,topcategory_s,endpointcategory_s,E.method_s,substanceType_s,publicname_s,reference_owner_s,Number of data points,endpointcategory_term,method_term
0,TOX,ENM_0000068_SECTION,MTS CELL VIABILITY ASSAY,silica nanoparticle,JRCNM02000a,FUNDP,148,http://www.bioassayontology.org/bao#ENM_0000068,http://purl.enanomapper.org/onto/ENM_8000273
1,TOX,ENM_0000068_SECTION,MTS CELL VIABILITY ASSAY,silica nanoparticle,JRCNM02000a,GAIKER,94,http://www.bioassayontology.org/bao#ENM_0000068,http://purl.enanomapper.org/onto/ENM_8000273
2,TOX,ENM_0000068_SECTION,MTS CELL VIABILITY ASSAY,silica nanoparticle,JRCNM02000a,IEM,43,http://www.bioassayontology.org/bao#ENM_0000068,http://purl.enanomapper.org/onto/ENM_8000273
3,TOX,ENM_0000068_SECTION,MTS CELL VIABILITY ASSAY,silica nanoparticle,JRCNM02000a,ANSES,30,http://www.bioassayontology.org/bao#ENM_0000068,http://purl.enanomapper.org/onto/ENM_8000273
4,TOX,ENM_0000068_SECTION,MTS CELL VIABILITY ASSAY,silica nanoparticle,JRCNM02000a,LEITAT,18,http://www.bioassayontology.org/bao#ENM_0000068,http://purl.enanomapper.org/onto/ENM_8000273


In [137]:
table = pd.pivot_table(df, values='Number of data points', index=['topcategory_s','endpointcategory_s','E.method_s'], columns=[ 'substanceType_s',"publicname_s"], aggfunc=np.sum).fillna("");
display(table.head())

table.reset_index().to_excel("method_material.xlsx")

substanceType_s                                                                         (14)-beta-D-glucan  \
publicname_s                                                                           BLEACHED BIRCH PULP   
topcategory_s endpointcategory_s    E.method_s                                                               
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                          
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                           
                                    OECD TG 201                                                              
                                    OECD TG 202                                                              
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                       

substanceType_s                                                                                        \
publicname_s                                                                           UPM BIRCH PULP   
topcategory_s endpointcategory_s    E.method_s                                                          
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                     
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                      
                                    OECD TG 201                                                         
                                    OECD TG 202                                                         
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                  

substanceType_s                                                                                                 \
publicname_s                                                                           UPM Bleached Birch Pulp   
topcategory_s endpointcategory_s    E.method_s                                                                   
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                              
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                               
                                    OECD TG 201                                                                  
                                    OECD TG 202                                                                  
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                           

substanceType_s                                                                        barium sulfate  \
publicname_s                                                                                   NM-220   
topcategory_s endpointcategory_s    E.method_s                                                          
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                     
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                      
                                    OECD TG 201                                                         
                                    OECD TG 202                                                         
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                  

substanceType_s                                                                        carbon nanotube  \
publicname_s                                                                                    NM-411   
topcategory_s endpointcategory_s    E.method_s                                                           
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                      
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                  

```
# try different excel writer
from StyleFrame import StyleFrame, Styler, utils
sf = StyleFrame(table.reset_index())
excel_writer=excel_writer = StyleFrame.ExcelWriter("method_material.xlsx")
sf.to_excel(excel_writer=excel_writer)
excel_writer=excel_writer.save()
```


In [138]:
table = pd.pivot_table(df, values='reference_owner_s', index=['topcategory_s','endpointcategory_s','E.method_s'], columns=[ 'substanceType_s','publicname_s'], 
                             aggfunc=lambda x: ' '.join(str(v) for v in x)).fillna("")

display(table.head())


substanceType_s                                                                         (14)-beta-D-glucan  \
publicname_s                                                                           BLEACHED BIRCH PULP   
topcategory_s endpointcategory_s    E.method_s                                                               
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                          
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                           
                                    OECD TG 201                                                              
                                    OECD TG 202                                                              
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                       

substanceType_s                                                                                        \
publicname_s                                                                           UPM BIRCH PULP   
topcategory_s endpointcategory_s    E.method_s                                                          
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                     
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                      
                                    OECD TG 201                                                         
                                    OECD TG 202                                                         
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                  

substanceType_s                                                                                                 \
publicname_s                                                                           UPM Bleached Birch Pulp   
topcategory_s endpointcategory_s    E.method_s                                                                   
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                              
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                               
                                    OECD TG 201                                                                  
                                    OECD TG 202                                                                  
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                           

substanceType_s                                                                        barium sulfate  \
publicname_s                                                                                   NM-220   
topcategory_s endpointcategory_s    E.method_s                                                          
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                     
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                      
                                    OECD TG 201                                                         
                                    OECD TG 202                                                         
                                    SOP-Toxicity Test with Daphnia magna (without NOM)                  

substanceType_s                                                                        carbon nanotube  \
publicname_s                                                                                    NM-411   
topcategory_s endpointcategory_s    E.method_s                                                           
ECOTOX        EC_ALGAETOX_SECTION   PSEUDOKIRCHNERIELLA SUBCAPITATA                                      
              EC_DAPHNIATOX_SECTION ACUTE TOXICITY                                                  

In [ ]:
table.reset_index().to_excel("method_material_provider.xlsx")